# Определение оценочной лексики с помощью Non-Negative  Matrix  Factorization

In [272]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from sklearn.externals import joblib
from sklearn import metrics
from tqdm import tqdm
from bs4 import BeautifulSoup
import gensim
from gensim.models import KeyedVectors
import re

## 1) Подготовка текстов

In [273]:
file = open('/Users/Valeriya/Downloads/SentiRuEval_rest_train.xml')

In [274]:
soup = BeautifulSoup(file, 'lxml')

In [275]:
all_texts = []

In [276]:
for txt in soup.find_all('text'):
    all_texts.append(txt.text)

## Preprocess data

In [278]:
raw_documents = []
snippets = []
for text in all_texts:
    raw_documents.append( text.lower() )
    snippets.append( text[0:min(len(text),100)] )
print("Read %d raw text documents" % len(raw_documents))

Read 19034 raw text documents


In [279]:
from nltk.tokenize import RegexpTokenizer
toker = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+')

In [280]:
stopwords = open('stop.txt').read()

In [281]:
stopwords = stopwords.split('\n')

In [282]:
# create BoW + tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(stop_words=stopwords, min_df=20)
A = vectorizer.fit_transform(raw_documents)
print( "Created %d X %d TF-IDF-normalized document-term matrix" % (A.shape[0], A.shape[1]) )

terms = vectorizer.get_feature_names()
joblib.dump((A,terms,snippets), "articles-tfidf.pkl") 

Created 19034 X 11425 TF-IDF-normalized document-term matrix


['articles-tfidf.pkl']

In [284]:
# # top features from tf-idf model
# import operator


# sums = np.array(A.sum(axis=0)).ravel()
# # map weights to the terms
# weights = { term: sums[col] for col, term in enumerate(terms)}
# ranking = sorted(weights.items(), key=operator.itemgetter(1), reverse=True)
# for i, pair in enumerate( ranking[0:20] ):
#     print( "%s (%.2f)" % ( pair[0], pair[1] ) )

## Apply NMF

In [285]:
(A,terms,snippets) = joblib.load( "articles-tfidf.pkl")

# create the model
k = 60
model = NMF( init="nndsvd", n_components=k ) 

W = model.fit_transform( A )
H = model.components_

In [286]:
tokenizer_tf = vectorizer.build_tokenizer()

In [288]:
# show topic descriptors
def get_descriptor(terms, H, topic_index, top):
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( terms[term_index] )
    return top_terms

descriptors = []
for topic_index in range(k):
    descriptors.append( get_descriptor( terms, H, topic_index, 20) )
    str_descriptor = ", ".join( descriptors[topic_index] )
    print("Topic %02d: %s" % ( topic_index+1, str_descriptor))

Topic 01: вкус, соус, блюдо, соусом, люблю, чай, вполне, вроде, мясо, поэтому, пюре, тар, правда, сразу, десерт, мяса, порция, соусе, наверное, внутри
Topic 02: кухня, отличная, понравилась, замечательная, вкусная, неплохая, прекрасная, японская, великолепная, итальянская, нравиться, отменная, отвратительная, высоте, хороша, порадовала, достойная, держать, европейская, неплохо
Topic 03: свадьбу, гости, огромное, свадьбы, гостей, зал, праздник, свадьба, банкета, торт, праздновали, банкет, благодарность, отмечали, отдельное, праздника, выразить, еды, хотим, восторге
Topic 04: принесли, кофе, счет, чай, заказали, официант, сразу, рублей, попросили, горячее, заказала, вместо, заказ, принести, сдачу, решили, итоге, официантка, заказал, оказалось
Topic 05: место, замечательное, прекрасное, идеальное, уютное, лучшее, отдыха, самое, единственное, романтического, любимое, посидеть, чудесное, хочется, приятное, приличное, поесть, встреч, достойное, проведения
Topic 06: понравилось, единственное,

In [289]:
from pymystem3 import Mystem
import collections
from collections import Counter, defaultdict
c = Counter()

In [290]:
m = Mystem()

In [291]:
lemmatized_descriptors = []
for arr in descriptors:
    for word in arr:
        lemma = m.lemmatize(word)[0]
        lemmatized_descriptors.append(lemma)

## 3) Получили дескрипторы. Теперь открываем w2v модель

In [292]:
vectors = KeyedVectors.load_word2vec_format('/Users/Valeriya/Desktop/ruscorpora_mean_hs.model.bin', binary=True)

In [293]:
## 4) basic_words - просто ручной список слов, которые на наш взгляд очень оценочно окрашены

In [294]:
basic_words = ['красивый_A', 'вкусный_A', 'прекрасный_A', 'качественный_A', 'ужасный_A', 'отвратительный_A',
              'мерзкий_A',  'советовать_V', 'рекомендовать_V', 'ужасно_ADV', 'нравиться_V', 'прекрасно_ADV', 'ужас_S',
              'радость_S', 'уютный_A', 'испорченный_A', 'несвежий_A', 'хороший_A', 'плохой_A']

In [295]:
mean = sum([vectors[i] for i in basic_words])/len(basic_words)

### mean - средний вектор по этим словам

In [296]:
words_with_pos = []
for word in lemmatized_descriptors:
    if word+'_S' in vectors:
        words_with_pos.append(word+'_S')
    elif word+'_A' in vectors:
        words_with_pos.append(word+'_A')
    elif word+'_V' in vectors:
        words_with_pos.append(word+'_V')
    elif word+'_ADV' in vectors:
        words_with_pos.append(word+'_ADV')
    elif word+'_APRO' in vectors:
        words_with_pos.append(word+'_APRO')
    else:
        print(word) # то, что не нашлось в модели - но там и не очень нужные слова

вроде
поэтому
вместо
оч
есть
14
запеченый
ура
много
поэтому
таки
15
20
40
30
45
25
сие
который
150
250
100
500
200
300
00
50
400
350
120
180
90
такой
какой
никто
вроде
один
такой
либо
поэтому
второй
такой
свой
никто
00
раз
250
второй
причем
10
12
11
07
он
мой
некоторые
такой
первый
какой
вроде
никто
раз
двое


In [297]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

## 5) Сравниваем каждое слово из words_with_pos с каждыс словом из basic_words, рассчитываем дальность (cosine distance),  если она меньше 0.9, вносим слово в список оценочных

In [298]:
final_list = []
count=0
from scipy.spatial import distance
for word in words_with_pos:
    for gold in basic_words:
        vec_descr = vectors[word]
        vec_gold= vectors[gold]
        
    if cosine(vec_descr, vec_gold) < 0.9:
        final_list.append(word)

In [300]:
len(set(final_list))

93

In [301]:
len(set(final_list)), len(set(words_with_pos))

(93, 484)

In [303]:
final = set(final_list)

In [305]:
final #непосредственно сам итоговый лист

{'адекватный_A',
 'благоприятный_A',
 'большой_A',
 'вежливый_A',
 'великолепный_A',
 'видимо_ADV',
 'вкус_S',
 'вкусно_ADV',
 'вкусный_A',
 'вообще_ADV',
 'всякий_APRO',
 'выбирать_V',
 'высокий_A',
 'главное_S',
 'гуманный_A',
 'демократичный_A',
 'доброжелательный_A',
 'добрый_A',
 'довольный_A',
 'дорого_ADV',
 'достойный_A',
 'дружелюбный_A',
 'единственный_A',
 'желать_V',
 'завышать_V',
 'замечательный_A',
 'здорово_ADV',
 'идеальный_A',
 'интересный_A',
 'испортить_V',
 'испорченный_A',
 'качественный_A',
 'качество_S',
 'классный_A',
 'комфортно_ADV',
 'красивый_A',
 'любимый_S',
 'мило_ADV',
 'молодой_A',
 'наверное_ADV',
 'навязчивый_A',
 'написать_V',
 'настроение_S',
 'необычный_A',
 'неплохо_ADV',
 'неплохой_A',
 'низкий_A',
 'новое_S',
 'нормальный_A',
 'нравиться_V',
 'обходительный_A',
 'общий_A',
 'обычный_A',
 'оказываться_V',
 'оригинальный_A',
 'отвратительный_A',
 'отзывчивый_A',
 'отличный_A',
 'отменный_A',
 'плохо_ADV',
 'плохой_A',
 'пожалеть_V',
 'положительн

# Итого получилось 93 слова

### 6) Находим коллокации слов из final_list

In [306]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [307]:
tokens = []
for arr in all_texts:
    words = word_tokenize(arr)
    for w in words:
        lemma = m.lemmatize(w)
        tokens.append(lemma)

In [308]:
tokens_clear = [arr[0] for arr in tokens]

In [309]:
finder = BigramCollocationFinder.from_words(tokens_clear)
# # only bigrams that appear 3+ times

In [325]:
words_for_filter = []
for word in set(final_list):
    word = re.sub('_.*', '', word)
#     creature_filter = lambda *w: word not in w
    words_for_filter.append(word)

In [311]:
def create_filter_minfreq_inwords(scored, words, minfreq):
    def bigram_filter(w1, w2):
        return (w1 not in words and w2 not in words) and (
                (w1, w2) in scored and scored[w1, w2] <= minfreq)
    return bigram_filter

In [312]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
scored = dict(finder.score_ngrams(bigram_measures.raw_freq))

In [313]:
myfilter = create_filter_minfreq_inwords(scored, words_for_filter, 5)

In [315]:
# sorted(words_for_filter)

In [316]:
finder.apply_ngram_filter(myfilter)
print(finder.nbest(bigram_measures.likelihood_ratio, 100))

[('в', 'общий'), ('день', 'рождение'), ('очень', 'понравиться'), ('молодой', 'человек'), ('очень', 'вкусно'), ('очень', 'вкусный'), ('оставаться', 'довольный'), ('высокий', 'уровень'), ('приятно', 'удивлять'), ('хотеться', 'бы'), ('очень', 'приятный'), ('мой', 'любимый'), ('всякий', 'похвала'), ('не', 'понравиться'), ('очень', 'довольный'), ('понравиться', '.'), ('не', 'пожалеть'), ('очень', 'хороший'), ('высоко', 'всякий'), ('не', 'хотеться'), ('самый', 'главное'), ('вкусно', ',\n'), ('общий', ',\n'), ('очень', 'нравиться'), ('вкусный', 'еда'), ('по', 'вкус'), ('на', 'вкус'), ('большой', 'количество'), ('я', 'понравиться'), ('оставлять', 'желать'), ('общий', 'впечатление'), ('добрый', 'день'), ('на', 'высокий'), ('вкусно', '.'), ('единственный', 'минус'), ('положительный', 'эмоция'), ('наверное', ',\n'), ('отличный', 'место'), ('.', 'единственный'), ('вкусный', ',\n'), ('очень', 'приятно'), ('большой', 'спасибо'), ('большой', 'компания'), ('не', 'плохой'), ('фоновый', 'музыка'), ('жел

## 7) Наконец, сравним наш список из 92 слов со словарем

In [317]:
sentilex = open('/Users/Valeriya/Downloads/rusentilex.txt').read()

In [318]:
sentilex_words = []
for item in sentilex.split('\n'):
    words = item.split(',')
    sentilex_words.append(words[0])

In [320]:
found_in_sentilex = []

In [328]:
for word in words_for_filter:
    if word in sentilex_words:
        found_in_sentilex.append(word)

In [331]:
len(set(found_in_sentilex))

49

# Совпадение с Сентилексом - 49 слов!

In [332]:
found_in_sentilex

['адекватный',
 'низкий',
 'удобный',
 'прекрасный',
 'радовать',
 'интересный',
 'добрый',
 'приемлемый',
 'оригинальный',
 'положительный',
 'приличный',
 'гуманный',
 'обходительный',
 'великолепный',
 'отвратительный',
 'стильный',
 'классный',
 'пожалеть',
 'красивый',
 'нормальный',
 'отзывчивый',
 'испортить',
 'идеальный',
 'довольный',
 'расторопный',
 'неплохой',
 'хороший',
 'завышать',
 'достойный',
 'ужасный',
 'доброжелательный',
 'замечательный',
 'приятный',
 'качественный',
 'нравиться',
 'шикарный',
 'отличный',
 'отменный',
 'демократичный',
 'вкусный',
 'умеренный',
 'плохой',
 'дружелюбный',
 'благоприятный',
 'навязчивый',
 'понравиться',
 'вежливый',
 'любимый',
 'чудесный']